In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Airline company has been providing high-quality air transportation service to our clients for several years
## Analysis of Airline company that is facing challenges due to several factors such rising fuel prices, increased interest rate, etc
## As a result, their company's profitability is under pressure, and they are seeking ways to address this issue
## To tackle this challenge, they are looking to conduct an analysis of their database to find ways to increase their occupancy rate, which can help boost average profit earned per seat


In [ ]:
## Goal: to identify oppurtunities to increase occupancy rate on low-performing flights
##       which can ultimately lead to increased protitability for the airline
## python will be used for data visulation and manipulation
## sql will fetch the data using queries 

In [ ]:
## importing libraries
import sqlite3 
import pandas as pd #for data manipulation 
import matplotlib.pyplot as plt  #for visulization 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## database connection using sqlite3


connection = sqlite3.connect('/kaggle/input/airline-data-analysis/travel.sqlite') 
cursor = connection.cursor() #to create query and execute it

In [ ]:
## seeing what types of tables in database
cursor.execute("select name from sqlite_master where type = 'table' ; ") #writing sequal query
print('List of tables present in the database')
table_list = [table[0] for table in cursor.fetchall()] #list comprehension
table_list

In [ ]:
## data exploration
## will see few tables one by one to see what type o columns presents and their types

aircrafts_data = pd.read_sql_query("select * from aircrafts_data", connection) #seeing first table and what values it contains
## used panda's read_sql_query for table output, unlike cursor above which gave values in list form
aircrafts_data.head() #head() j shows first 5

In [ ]:
## seeing how many rows mentioned which will tell how many aircrafts
aircrafts_data.shape # shows 9 present

In [ ]:
aircrafts_data ## seeing all 9 aircrafts (rows) data

In [ ]:
## seeing bookings data
bookings = pd.read_sql_query("select * from bookings", connection) 
bookings

In [ ]:
## seeing airports_data
airports_data = pd.read_sql_query("select * from airports_data", connection) 
airports_data
## by looking at table we see we have info in english and russian, need to only keep understandable data

In [ ]:
## seeing data types of columns in all different tables
for table in table_list:
    print("\n table:", table) # just places heading of each section/table 
    
    column_info = connection.execute("PRAGMA table_info({})".format(table)) ## PRAGMA used whenever we want info of table
    for column in column_info.fetchall():
        print(column[1:3])

In [ ]:
## checking for missing values in tables
for table in table_list:
    print ("\n table:", table)
    df_table = pd.read_sql_query(f"select * from {table}", connection)
    print(df_table.isnull().sum())

In [ ]:
## the many zeors indiciate there is no missing value, data is consistent
## no cleanign of data needed

In [ ]:
## our analysis will be on 3 question to tell the client more about their data and give meaningful insight about their data that may be important for their business

## 1) how many planes have more than 100 seats ... will tell poplr. flights
## 2) how the number of tickets booked and total amount earned changed with the time
## 3)calculate the average charges for each aircraft with different fare conditions

In [ ]:
## exploring first q --> how many planes have more than 100 seats
pd.read_sql_query("select aircraft_code, count(*) as num_seats from seats group by aircraft_code having num_seats > 100", connection)
## out of 9, 6 have more than 100 seats

In [ ]:
## with this basic analysis we see that 6/9 planes have more than 100 seats
## by using larger data set we can find a more accurate finding for our next two questions
## will be useful in developing strategies to increase occupancy rates and optimize pricing for each aircraft

In [ ]:
## exploring second q --> ticket booked/amount changed with time
## will use line graph to show (bc good to show change with time)


tickets = pd.read_sql_query("select * from tickets inner join bookings on tickets.book_ref = bookings.book_ref", connection)

tickets['book_data'] = pd.to_datetime(tickets['book_date'])
tickets.dtypes

In [ ]:
tickets = pd.read_sql_query("select * from tickets inner join bookings on tickets.book_ref = bookings.book_ref", connection)

tickets['book_data'] = pd.to_datetime(tickets['book_date'])
tickets['date'] = tickets['book_data'].dt.date # creating new column with dates
tickets

In [ ]:
## using groupby to group by date then counting how many tickets booked that day
## will return in sorted order (ascending order)
tickets.groupby('date')[['date']].count()

In [ ]:
## creating the line graph
x = tickets.groupby('date')[['date']].count()
plt.figure(figsize = (18,6))
plt.plot(x.index, x['date'], marker = '^')
plt.xlabel('Date', fontsize = 20)
plt.ylabel('Number of Tickets', fontsize = 20)
plt.grid('b')
plt.show()

## we see theres a higher spike in etc etc 

In [ ]:
## doing same for bookings
## the plots will be similar but it may show small spikes we couldn't see in previous graph
bookings = pd.read_sql_query("select * from bookings", connection)
bookings['book_data'] = pd.to_datetime(bookings['book_date'])
bookings['date'] = bookings['book_data'].dt.date

In [ ]:
## grouping by sum of total amount($) of bookings booked by date .. earnings of airline 
bookings.groupby('date')[["total_amount"]].sum()

In [ ]:
x = bookings.groupby('date')[["total_amount"]].sum()
plt.figure(figsize = (18,6))
plt.plot(x.index, x['total_amount'], marker = '^')
plt.xlabel('Date', fontsize = 20)
plt.ylabel('Total amount earned', fontsize = 20)
plt.grid('b')
plt.show()

In [ ]:
## Observations:
## upon analysis of the chart we observe that # of ticket book increase from June 22 to July 7
## this increased is followed by a stable pattern from July 8th until August with a noticible peak in ticket booking where highest number was bought on a single day
## revenue earned is closely tied with booking (possibly not much other income source for company)
## therefore we see similar trend in total revenue and booking through the analyzed time period
## we can explore other factors contributing to the peak in ticket bookings could help increase overall revenue and optimize operational strategies

In [ ]:
## exploring third q --> average charge for each aircraft w/ diff. fare conditions

df = pd.read_sql_query("select fare_conditions, aircraft_code, avg(amount) from ticket_flights join flights on ticket_flights.flight_id = flights.flight_id group by aircraft_code, fare_conditions", connection)
df

In [ ]:
## using bar plot to compare business/economic flights
sns.barplot(data = df, x = 'aircraft_code', y = 'avg(amount)', hue = 'fare_conditions')

In [ ]:
## This bar graph is made after we computed average costs associated with different fare conditions for each aircraft
## The bar graph shows data types of 3 fares: business, economy and comfort
## Note comfort class is only available for one aircraft (number 773)
## Also note CN1 and CR2 only provide economy 
## comparing prices of all aircrafts we see the charges for business are consistently greater then economy regardless of fare condition of planes


In [ ]:
## The highest total revenue is generated by the SU9 aircraft, likely due to its lower ticket prices. 
## The CN1 aircraft has the lowest total revenue, possibly due to its limited economy class offering. 
## Monitoring average occupancy rates helps airlines fill seats efficiently, increase revenue, and reduce expenses. 
## Improving occupancy rates can be financially beneficial and achieved through pricing strategies and operational considerations. 

In [ ]:
## Examining Occupancy Rate

## airlines should analyze their revenue streams in order to maximize profitability 
## we can see income per year/average revenue per ticket for each aircraft to determine which aircraft types generate more income and alter operations appropriately 
## this can help identify potential for pricing optimization and allocating resources to more profitable routes 

## analyzing occupancy rate questions to consider
## Q1: for each air craft calculate total revenue per year and average revenue per ticket
## Q2: calculate average occupancy per aircraft
## Q3: calculate by how much total annual turnover could increase by giving all aircraft a 10% higher occupancy rate


In [ ]:
## total revenue per year and average revenue per ticket
pd.read_sql_query("select aircraft_code, ticket_count, total_revenue, total_revenue/ticket_count as avg_revenue_per_ticket from (select aircraft_code, count(*) as ticket_count, sum(amount) as total_revenue from ticket_flights join flights on ticket_flights.flight_id = flights.flight_id group by aircraft_code)", connection)

In [ ]:
## the aircraft with highest total revenue is SU9 (from a previous graph we see business and economy tickets are cheapest for SU9)
## low price = more people buying = higher revenue 

## the least total revenue earned was by CN1. This may be because it only offers economy class at a super cheap (possibly due to poor conditions or facilities so people dont buy again)


In [ ]:
## calculate average occupancy per aircraft
occupancy_rate = pd.read_sql_query("""select a.aircraft_code,avg(a.seats_count) as booked_seats, b.num_seats, avg(a.seats_count)/b.num_seats as occupancy_rate from
                (select aircraft_code,flights.flight_id,count(*) as seats_count from boarding_passes
                    inner join flights
                    on boarding_passes.flight_id = flights.flight_id
                    group by aircraft_code,flights.flight_id) as a
                    inner join 
                    (select aircraft_code,count(*) as num_seats from seats
                    group by aircraft_code) as b
                    on a.aircraft_code = b.aircraft_code group by a.aircraft_code""", connection)
occupancy_rate

In [ ]:
## through average occupancy rate, airlines can measure how successfully their fill their seats and find ways to boost occupancy rate with this
## higher occupancy rates can help airlines increase revenuw and proftability while lowering operational expenses associated with vacant seats 

## occupancy rate is calculated by dividing booked seats by total number of seats 

## higher occupancy rate means aircraft seats are more booked

In [ ]:
## calculating how much the total annual turnover would increase by giving all aircraft a 10% higher occupancy rate
occupancy_rate['Inc occupancy rate'] = occupancy_rate['occupancy_rate'] + occupancy_rate['occupancy_rate']*0.1
occupancy_rate

In [ ]:
pd.set_option("display.float_format",str)

In [ ]:
## airlines can assess how their total yearly turnover could improve by providing aircraft 10% higher occupancy rate
## this can help further examine possible benefits of raising occupancy rates and tell if this financial strategy is worthwile 
## above table shows how total revenue increased after increasing occupancy rate by 10%, it also shows a gradual increase so airlines should be more focused on pricing strategy 

In [ ]:
total_revenue = pd.read_sql_query("""select aircraft_code,sum(amount) as total_revenue from ticket_flights
                        join flights on ticket_flights.flight_id = flights.flight_id
                        group by aircraft_code""", connection)
total_revenue


In [ ]:
occupancy_rate['Inc Total Annual Turnover'] = (total_revenue['total_revenue']/occupancy_rate['occupancy_rate'])*occupancy_rate['Inc occupancy rate']
occupancy_rate

In [ ]:
# Summary

In [ ]:
## Factors such as total revenue, average revenue per ticket, and average occupancy per aircraft play a crucial role in this analysis.
## greater occupancy rate is one important feature that can enhance profitability since it allows airlines to maximize revenuw while minimizing costs associated with vacant seats
## prices should also be revised as high prices tickets are not popular due to the cost, nor are low cost tickets as the facilities provided by low cost may be poor 
## boosting occupancy rates should not come at price of consumer happiness or safety .. it's important to consider consumer happiness, offer high-quality service and upholding safety regulations while striving for profit
## Balancing these factors is key to long-term success in the competitive airline industry